# IA - Modèle CNN - BDD signaux acceptables

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import Fonctions_BDD_Signaux_Artificiels as BDD
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras import datasets, layers, models

from scipy import signal
import scipy.io.wavfile as wav
%matplotlib widget

In [2]:
liste_comp = BDD.BDD_type('A')

liste_train = []
liste_test = []

for i in range(1,229):
    if i % 4 == 0 :
        liste_test.append(liste_comp[i-1])
    else :
        liste_train.append(liste_comp[i-1])

print("Liste complète", len(liste_comp))
print("75%", len(liste_train))
print("25%", len(liste_test))

Liste complète 228
75% 171
25% 57


### Définition des labels

In [3]:
target_train = []
target_test = []

for i in range(0,43):
    target_train.append('Gendarmerie')
for i in range(43,86):
    target_train.append('Police')    
for i in range(86,129):
    target_train.append('Pompiers')
for i in range(129,171):
    target_train.append('Samu')

for i in range(0,14):
    target_test.append('Gendarmerie')
for i in range(14,28):
    target_test.append('Police')
for i in range(28,42):
    target_test.append('Pompiers')
for i in range(42,57):
    target_test.append('Samu')

print(len(target_train))
print(len(target_test))

171
57


### Mélange de la BDD

In [4]:
liste_test_s, target_test_s = shuffle(liste_test, target_test)
liste_train_s, target_train_s = shuffle(liste_train, target_train)

### Définition de image_train et image_test

In [5]:
image_train = BDD.return_image(liste_train_s, 'C:/Users/abras/OneDrive - etu.sorbonne-universite.fr/M1/Projet/Projet_final/Sirenes artificielles')
image_test = BDD.return_image(liste_test_s, 'C:/Users/abras/OneDrive - etu.sorbonne-universite.fr/M1/Projet/Projet_final/Sirenes artificielles')
image_train = image_train[...,None]#Etape nécessaire à la detection des couleurs par les couches conv2D
image_test = image_test[...,None]#Same
print(image_train.shape)
print(image_test.shape)

(171, 41, 27, 1)
(57, 41, 27, 1)


### Encodage des label_train et label_test

In [6]:
encodage = {"Gendarmerie":0, "Police":1, "Pompiers":2, "Samu":3}
sirenes_labels_numeriques_train = []
for label in target_train_s :
    sirenes_labels_numeriques_train.append(encodage[label])

sirenes_labels_numeriques_test = []
for label in target_test_s :
    sirenes_labels_numeriques_test.append(encodage[label])

In [7]:
labels_train_onehot = tf.keras.utils.to_categorical(sirenes_labels_numeriques_train)
labels_test_onehot = tf.keras.utils.to_categorical(sirenes_labels_numeriques_test)

print(labels_train_onehot.shape)
print(labels_test_onehot.shape)

(171, 4)
(57, 4)


### IA test

In [8]:
# On convertit nos valeurs en float

image_train = image_train.astype(float)
image_test = image_test.astype(float)

# On normalise les pixels pour avoir des valeurs entre 0 et 1

image_train = image_train/255
image_test = image_test/255

In [9]:
model = tf.keras.models.Sequential()

In [10]:
# model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=(41, 27, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(32, (3,3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (1, 1), activation='relu'))

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(41, 27, 1)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (1, 1), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (1, 1), activation='relu'))

In [11]:
model.add(tf.keras.layers.Flatten())

In [12]:
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(4,activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 25, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 8, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 7, 32)         4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 3, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 3, 64)          2112      
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               1

In [14]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [15]:
history = model.fit(image_train, labels_train_onehot,
                    epochs=20,
                    validation_split=0.1)

Epoch 1/20
5/5 [==============================] - 2s 274ms/step - loss: 10.4290 - accuracy: 0.2842 - val_loss: 13.3004 - val_accuracy: 0.2778
Epoch 2/20
5/5 [==============================] - 0s 56ms/step - loss: 8.6353 - accuracy: 0.2347 - val_loss: 2.3587 - val_accuracy: 0.2778
Epoch 3/20
5/5 [==============================] - 0s 57ms/step - loss: 1.9822 - accuracy: 0.3290 - val_loss: 1.6486 - val_accuracy: 0.5000
Epoch 4/20
5/5 [==============================] - 0s 57ms/step - loss: 1.9823 - accuracy: 0.3125 - val_loss: 1.4916 - val_accuracy: 0.3333
Epoch 5/20
5/5 [==============================] - 0s 56ms/step - loss: 1.1675 - accuracy: 0.5037 - val_loss: 1.8015 - val_accuracy: 0.2222
Epoch 6/20
5/5 [==============================] - 0s 56ms/step - loss: 1.1204 - accuracy: 0.5333 - val_loss: 1.4388 - val_accuracy: 0.5556
Epoch 7/20
5/5 [==============================] - 0s 59ms/step - loss: 0.9514 - accuracy: 0.5946 - val_loss: 1.2911 - val_accuracy: 0.5000
Epoch 8/20
5/5 [========

In [16]:
loss_curve = history.history["loss"]
acc_curve = history.history["accuracy"]

loss_val_curve = history.history["val_loss"]
acc_val_curve = history.history["val_accuracy"]

plt.figure()
plt.plot(loss_curve, label="Train")
plt.plot(loss_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.figure()
plt.plot(acc_curve, label="Train")
plt.plot(acc_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Accuracy")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
loss, acc = model.evaluate(image_test, labels_test_onehot)
print("Test Loss", loss)
print("Test Accuracy", acc)

2/2 [==============================] - 0s 7ms/step - loss: 0.8027 - accuracy: 0.7544
Test Loss 0.8027406930923462
Test Accuracy 0.7543859481811523
